# BMI/CS 576 Fall 2023 - HW4
The objectives of this homework are to practice with

* weighted parsimony
* tree space search
* Markov chains


## HW policies
Before starting this homework, please read over the [homework policies](https://canvas.wisc.edu/courses/374201/pages/hw-policies) for this course.  In particular, note that homeworks are to be completed *individually* and plagiarism from any source (with the one exception noted below) will be considered **academic misconduct**.

You are welcome to use any code from the weekly notebooks (including the official solutions) in your solutions to the HW.

## Modules for this HW

In [1]:
import toytree
import fasta
import submatrix

In [2]:
import math

## PROBLEM 1: Weighted parsimony (40 points)

In this problem you will implement the [weighted parsimony algorithm](https://canvas.wisc.edu/courses/374201/pages/day-16-online-lecture-phylogenetic-trees-weighted-parsimony) (also see [worked example](https://canvas.wisc.edu/courses/374201/modules/items/6356958)) for computing the cost of a tree given assignments of characters to its leaves.  Implement this algorithm as a function `weighted_parsimony` below, which takes as input a tree, a dictionary giving the character at each leaf, a cost matrix, and a string specifying all valid characters (e.g., "ACGT").  Your function will output the minimum cost of the tree as well as a dictionary giving an assignment of characters to all nodes of the tree (including the leaves) that achieves the minimum cost.

### Implementation details

* In the case of a tie during the traceback (e.g., multiple characters at a child node that allow for the same minimum cost at the parent, or multiple characters at the root node that give the same minimum cost), pick the character that is lexicographically smallest.

* You may find useful the form of the implementations of `fitch_score_and_min_cost_states` and `fitch_ancestral_states` in the Day 16 notebook for the fill and traceback stages, respectively.

* To directly access the root node of a `toytree.tree` object, simply access its `treenode` attribute. 

Tests for Problem 1 are found at the bottom of this notebook.

### Cost matrices to be used in this assignment

In [3]:
DNA = "ACGT"
basic_dna_cost_matrix = submatrix.match_mismatch_matrix(0, 1, DNA)
purine_pyrimidine_cost_matrix = submatrix.read_substitution_matrix("purine_pyrimidine.txt")

print("basic_dna_cost_matrix = ")
submatrix.print_matrix(basic_dna_cost_matrix)
print("purine_pyrimidine_cost_matrix = ")
submatrix.print_matrix(purine_pyrimidine_cost_matrix)

basic_dna_cost_matrix = 
         A    C    G    T
    A    0    1    1    1
    C    1    0    1    1
    G    1    1    0    1
    T    1    1    1    0
purine_pyrimidine_cost_matrix = 
         A    C    G    T
    A    0    2    1    2
    C    2    0    2    1
    G    1    2    0    2
    T    2    1    2    0


In [4]:
def weighted_parsimony(tree, leaf_states, cost_matrix, alphabet = DNA):
    """Computes the minimum cost of a tree and an assignment of ancestral characters achieving that cost.   
    Args:
        tree: a toytree tree.
        leaf_states: a dictionary mapping leaf names to characters.
        cost_matrix: a cost matrix (represented as a dictionary with tuples as keys)
           where cost_matrix[a, b] is the cost of a substitution between characters a and b
        alphabet: a string specifying the possible character states that each node may take.
    Returns:
        A tuple (min_cost, node_states) where min_cost is the minimum cost of the tree (a numeric value)
        and node_states is an assignment of characters to the nodes that achieves this minimum cost, 
        (a dictionary mapping node names to characters).
    """      
    
    score = [0,0,0,0]
    # print(cost_matrix)
    R = {}
    #find minimum score
    for node in tree.treenode.traverse("postorder"):
        if node.is_leaf():
            R[node.name] = {leaf_states[node.name]}
        else:
            left, right = [R[child.name] for child in node.children]
            states = left & right
            if states:
                R[node.name] = states
                index = 0
                if len(left) == 1:
                    for i,j in cost_matrix:
                        for k in left:
                            if i == k:
                                score[index] += cost_matrix[(i,j)]
                                index += 1

            else:
                R[node.name] = left | right
                minimum = math.inf
                maximum = 0
                if len(right) == 1:
                    index = 0
                    for i,j in cost_matrix:
                        for k in right:
                            if i == k:
                                score[index] += cost_matrix[(i,j)]
                                index += 1
                index = 0
                if len(left) == 1:
                    for i,j in cost_matrix:
                        for k in left:
                            if i == k:
                                score[index] += cost_matrix[(i,j)]
                                index += 1
     
    #find r   
    r = {} 
    for node in tree.treenode.traverse("preorder"):
        if node.is_root():
            find = math.inf
            index = 0
            for i in range(len(score)):
                if score[i] < find:
                    find = score[i]
                    index = i
            r[node.name] = sorted(alphabet)[index]
        else:
            p = node.up
            if r[p.name] not in R[node.name]:
                if len(R[node.name]) == 1:
                    r[node.name] = sorted(R[node.name])[0]
                else:
                    find = math.inf
                    index = 0
                    for i in range(len(score)):
                        if score[i] < find:
                            find = score[i]
                            index = i
                    r[node.name] = sorted(alphabet)[index]
            else:
                r[node.name] = r[p.name]
    # print(score, min(score), r)
    return min(score), r

## Helper functions

You may find the function below helpful for visualizing trees and the names of its nodes.

In [5]:
def draw_tree_with_internal_labels(t, leaf_states=None):
    """Draws the given toytree tree with all nodes labeled and 
    (optionally) with character assignments to the leaves.
    Args:
        tree: a toytree tree.
        leaf_states: a dictionary mapping leaf names to characters.
    """
    if leaf_states:
        tip_labels = [leaf_states[leaf_name] for leaf_name in t.get_tip_labels()]
    else:
        tip_labels = False
    t.draw(node_labels=t.get_node_values(feature="name", show_root=True, show_tips=True),
           tip_labels=tip_labels,
           node_sizes=20,
           use_edge_lengths=False)

# Example usage:
t = toytree.tree("(Z,(X,Y));")
leaf_states = {"X": "C", "Y": "T", "Z": "A"}
draw_tree_with_internal_labels(t, leaf_states)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tac748e935a7e46e39ee45dd1d1daf5a1"> Y X Z 3 4 T C A

## PROBLEM 2: Human coronavirus phylogeny (20 POINTS)

There are many types of coronaviruses that can infect humans, some of which are quite common and cause a subset of common cold cases.  It is helpful to understand the phylogenetic relationships and between these viruses and their related evolutionary histories.  Included with this assignment is a multiple sequence alignment (`human_coronavirus_rdrp.fasta`) of the RNA-dependent RNA polymerase (RdRP) gene from six human coronavirus genomes (SARS-CoV-2, SARS_CoV_1, MERS, OC43, HKU1, and CoV229E).  For simplicity and efficiently, all gapped columns and all uninformative columns (no differences within the column) have been removed from this alignment.

In this problem, we will use your `weighted_parsimony` function from problem 1 to analyze the evolutionary relationships between these viruses.  If your `weighted_parsimony` function is incorrect, you may use the unweighted parsimony algorithms implemented in the Day 16 notebook instead.

**(a)** Using your `weighted_parsimony` function, compute the weighted parsimony score for all possible rooted trees of these six viral genomes, assuming CoV229E is the outgroup.  For your convenience, the file `all_rooted_trees.txt` contains the newick strings for all such trees.  Use `purine_pyrimidine_cost_matrix` as the cost matrix for your weighted parsimony computations.  *Hint: You will likely want to use slightly modified versions of functions and code from the Day 16 notebook for this problem*.

In [6]:
def alignment_leaf_states_list(alignment, sequence_names):
    """Returns a list of dictionaries, where each dictionary corresponds to the leaf states
    for a column of the alignment."""
    return [dict(zip(sequence_names, column)) for column in zip(*alignment)]

def score_tree_parsimony(tree, alignment, sequence_names):
    """Computes the parsimony score for a given tree and alignment.
    
    Args:
        tree: a toytree tree object
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        The parsimony score (a number)
    """
    fitch = []
    # column_scores, column_Rs = zip(*fitch_results)
    for i in alignment_leaf_states_list(alignment, sequence_names):
        fitch += [weighted_parsimony(tree, i, purine_pyrimidine_cost_matrix)]
    a, b = zip(*fitch)
    return sum(a)

In [7]:
ans = []
leafs = fasta.read_sequences_from_fasta_file("human_coronavirus_rdrp.fasta")
leaf = {}
for name, val in leafs:
    leaf[name] = val

with open("all_rooted_trees.txt") as f:
    treeList = f.read().splitlines()

     
for i in treeList:
    tree = toytree.tree(i)
    # tree.root(wildcard="CoV229E")
    # print(score_tree_parsimony(tree,leaf.values(),leaf.keys()), i)
    ans.append((score_tree_parsimony(tree,leaf.values(),leaf.keys()), i))
    
ans

[(5906, '(CoV229E,((((SARS-CoV-2,SARS_CoV_1),MERS),OC43),HKU1));'),
 (6293, '(CoV229E,((((SARS-CoV-2,MERS),SARS_CoV_1),OC43),HKU1));'),
 (6304, '(CoV229E,(((SARS-CoV-2,(SARS_CoV_1,MERS)),OC43),HKU1));'),
 (5618, '(CoV229E,((((SARS-CoV-2,SARS_CoV_1),OC43),MERS),HKU1));'),
 (5983, '(CoV229E,((((SARS-CoV-2,OC43),SARS_CoV_1),MERS),HKU1));'),
 (5987, '(CoV229E,(((SARS-CoV-2,(SARS_CoV_1,OC43)),MERS),HKU1));'),
 (5518, '(CoV229E,(((SARS-CoV-2,SARS_CoV_1),(MERS,OC43)),HKU1));'),
 (5629, '(CoV229E,((((SARS-CoV-2,MERS),OC43),SARS_CoV_1),HKU1));'),
 (5666, '(CoV229E,((((SARS-CoV-2,OC43),MERS),SARS_CoV_1),HKU1));'),
 (5840, '(CoV229E,(((SARS-CoV-2,(MERS,OC43)),SARS_CoV_1),HKU1));'),
 (5535, '(CoV229E,(((SARS-CoV-2,MERS),(SARS_CoV_1,OC43)),HKU1));'),
 (5554, '(CoV229E,(((SARS-CoV-2,OC43),(SARS_CoV_1,MERS)),HKU1));'),
 (5649, '(CoV229E,((SARS-CoV-2,((SARS_CoV_1,MERS),OC43)),HKU1));'),
 (5673, '(CoV229E,((SARS-CoV-2,((SARS_CoV_1,OC43),MERS)),HKU1));'),
 (5848, '(CoV229E,((SARS-CoV-2,(SARS_CoV_1,(MERS

**(b)** List the scores and trees for the **3** trees with the smallest weighted parsimony scores.  Given that these trees have similar scores, for what aspect of the true tree is there the most uncertainty?

In [8]:
sortTree = sorted(ans, key=lambda x: x[0])

sortTree[:3] 

[(5470, '(CoV229E,(((SARS-CoV-2,SARS_CoV_1),(OC43,HKU1)),MERS));'),
 (5483, '(CoV229E,(((SARS-CoV-2,MERS),OC43),(SARS_CoV_1,HKU1)));'),
 (5487, '(CoV229E,((SARS-CoV-2,OC43),((SARS_CoV_1,HKU1),MERS)));')]

**(c)** For the tree with the smallest score, use the results of your `weighted_parsimony` function to construct the ancestral sequence at the root node of the tree.  Save this sequence to a file `ancestor.fasta` in FASTA format. *Hint: You will likely want to use slightly modified versions of functions and code from the Day 16 notebook for this problem*

In [9]:
def compute_ancestral_sequences(tree, alignment, sequence_names):
    """Computes (reconstructed) sequences for all nodes in the given tree, given a multiple alignment.
    
    Args:
        tree: a toytree tree object
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        A dictionary mapping node names to strings (reconstructed sequences).
    """
    for i,j in sortTree:
        tree = toytree.tree(j)
        fitch_results = [weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
                         for leaf_states in alignment_leaf_states_list(alignment, sequence_names)]
        column_scores, column_Rs = zip(*fitch_results)
        node_names = [node.name for node in tree.treenode.traverse("postorder")]
        full_alignment = [''.join([states[name] for states in column_Rs]) for name in node_names]
    return {name: sequence for name, sequence in zip(node_names, full_alignment)}

In [10]:
first_sequences = compute_ancestral_sequences(sortTree[0],leaf.values(),leaf.keys())

In [11]:
def write_sequences_to_fasta_file(sequences, filename, wrap_width=70):
    """Writes all sequence records in the list sequences
    to the file specified by filename in FASTA format.
    Each record should be represented by a tuple (name, sequence)"""
    with open(filename, 'w') as f:
        for name, sequence in sequences.items():
            print(">" + name, file=f)
            print(wrap_sequence(sequence, wrap_width), file=f)
            
def wrap_sequence(s, width):
    return "\n".join(s[start: start + width] for start in range(0, len(s), width))

In [12]:
file_name = "ancestor.fasta"
write_sequences_to_fasta_file(first_sequences,file_name)

## PROBLEM 3: Branch and Bound with unweighted Parsimony (25 points)

Suppose we wish to find an unrooted tree with the minimum unweighted parsimony score for five taxa: 1,2,3,4,and 5, which have character states $A, C, C, A, C$, respectively.  In this problem, we will use the first branch and bound method described in the [Day 16 Tree space search lecture](https://canvas.wisc.edu/courses/374201/pages/day-16-online-lecture-phylogenetic-trees-tree-space-search) (slide 7: "Exact Method: Branch and Bound") to find such a tree.  We will use the unweighted parsimony score of a partial tree as the lower bound for the score of a full tree that may be built from it.

**(a)** Manually run the branch and bound algorithm on these data starting with the unrooted tree containg taxa 1, 2, and 3.  At the end of each iteration of the algorithm, list the elements (as newick strings) of the queue with their lower bounds. You do *not* need to show your work with respect to computing the parsimony score of each (partial) tree.

**(b)** For how many (partial) trees did you have to compute a parsimony score during the algorithm in part (a)?  How does this compare to the number of possible unrooted trees of five taxa?

###
### Your solution to Problem 3a here
###


**(A)**

Round 1:
(1,2,3) 

Round2 : (1,2,3) -> (1,2,3,4), (1,2,3,5)\
Q : [(1,2,3,4), (1,2,3,5)]

i) (1,2,3,4)\
tree: ((1,4),2,3) score = 1\
tree: ((1,2),3,4) score = 2\
tree: ((1,3),2,4) score = 2

ii) (1,2,3,5)\
tree: ((1,2),3,5) score = 1\
tree: ((1,3),2,5) score = 1\
tree: ((1,5),2,3) score = 1


Round3: (1,2,3,4) -> (1,2,3,4,5)\
Q : [(1,2,3,4,5)]

tree: ((1,4),(2,5),3) score = 1\
tree: ((1,4),(3,5),2) score = 1\
tree: (((1,4),5),2,3) score = 1\
tree: (((1,5),4),2,3) score = 2\
tree: (((4,5),1),2,3) score = 2


###
### Your solution to Problem 3b here
###


**(b)**

I made 11 tree to to compute a parsimony score during the algorithm in part a.

the number of possible unrooted trees of five taxa -> (2n-5)!!\
(2*5 -5)!! = (10 -5)!! = (5)!! = 5 * 3 * 1 = 15

## PROBLEM 4: Markov chain parameter estimation and likelihood (15 points)

Suppose we are given the following five DNA sequences. In this problem, we will model these of sequences using a simple Markov chain with a state for each of the four DNA bases.

$\begin{eqnarray}
x_1 & = & \mathrm{\tt ATGT} \\
x_2 & = & \mathrm{\tt AAAA} \\
x_3 & = & \mathrm{\tt GTCG} \\
x_4 & = & \mathrm{\tt AACA} \\
x_5 & = & \mathrm{\tt TACC} \\
\end{eqnarray}$

**(a)** Using uniform distributions for the transition probabilities and initial state probabilities, calculate the likelihood, $P(x_1, x_2, x_3, x_4, x_5)$, of these sequences (where we assume that each sequence is generated independently from the model).

**(b)** Estimate the parameters (transition and initial probabilities) of the Markov chain using maximum likelihood estimates. Calculate the likelihood of these sequences given these maximum likelihood parameter estimates.

**(c)** Estimate the parameters (transition and initial probabilities) of the Markov chain using Laplace estimates (pseudocount = 1). Calculate the likelihood of these sequences given these Laplace parameter estimates.

###
### Your solution to Problem 4a here
###


**(a)**

𝑃(𝑥1,𝑥2,𝑥3,𝑥4,𝑥5) = p(x1) * p(x2|x1) * p(x3|x2) * p(x4|x3) * p(x5|x4) = $(1/4)^4$

###
### Your solution to Problem 4b here
###


**(b)**

P(x1)= Count(x1)/Total Sequences

pr(a) = 9/20 = 0.45\
pr(c) = 4/20 = 0.2\
pr(g) = 3/20 = 0.15\
pr(t) = 4/20 = 0.2


###
### Your solution to Problem 4c here
###


**(c)**

P(x1)= Count(x1)+1/Total Sequences+4

pr(a) = 9+1/20+4 = 10/24 = 0.4167\
pr(c) = 4+1/20+4 = 5/24 = 0.208\
pr(g) = 3+1/20+4 = 4/24 = 0.1667\
pr(t) = 4+1/20+4 = 5/24 = 0.208

## Tests for Problem 1

In [13]:
pair_tree =              toytree.tree("(X,Y);")
triple_tree =         toytree.tree("(Z,(X,Y));")
quartet_tree =     toytree.tree("(W,(Z,(X,Y)));")
quartet2_tree =   toytree.tree("((W,Z),(X,Y));")
quintet_tree = toytree.tree("((V,W),(Z,(X,Y)));")
large_tree = toytree.tree("((E,(F,(G,H))),((A,B),(C,D)));")

pair_match_states =                             {"X": "C", "Y": "C"}
pair_mismatch_states =                          {"X": "A", "Y": "T"}
triple_states =                       {"Z": "C", "X": "A", "Y": "T"}
quartet_states =            {"W": "A", "Z": "C", "X": "A", "Y": "T"}
quartet2_states =           {"W": "C", "Z": "C", "X": "A", "Y": "T"}
quintet_states =  {"V": "C", "W": "C", "Z": "C", "X": "A", "Y": "T"}
large_states = {"A": "G", "B": "C", "C": "T", "D": "A", "E": "G", "F": "C", "G": "C", "H": "C"}

In [14]:
# pair_match_score (6 points)
tree, leaf_states = pair_tree, pair_match_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 0
print("SUCCESS: pair_match_score test case passed!")

SUCCESS: pair_match_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="teed577fc43434f85b54a916036440636"> Y X 2 C C

In [15]:
# pair_mismatch_score (6 points)
tree, leaf_states = pair_tree, pair_mismatch_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 2
print("SUCCESS: pair_mismatch_score test case passed!")

SUCCESS: pair_mismatch_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t22a95a52d5f04776ac9ec12c8ceadd3e"> Y X 2 T A

In [16]:
# triple_score (4 points)
tree, leaf_states = triple_tree, triple_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 3
print("SUCCESS: triple_score test case passed!")

SUCCESS: triple_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te247dd526b084d39801e397ebad95026"> Y X Z 3 4 T A C

In [17]:
# quartet_score (2 points)
tree, leaf_states = quartet_tree, quartet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 4
print("SUCCESS: quartet_score test case passed!")

SUCCESS: quartet_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta3378a2ce99e487cb0622ac935170cc9"> Y X Z W 4 5 6 T A C A

In [18]:
# quartet2_score (2 points)
tree, leaf_states = quartet2_tree, quartet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 4
print("SUCCESS: quartet2_score test case passed!")

SUCCESS: quartet2_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taad3cc6a1859445fad7141dd39050fe5"> Y X Z W 4 5 6 T A C A

In [19]:
# quintet_score (1 points)
tree, leaf_states = quintet_tree, quintet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 3
print("SUCCESS: quintet_score test case passed!")

SUCCESS: quintet_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tee7a61cf7c424ac985bdcae50ca39868"> Y X Z W V 5 6 7 8 T A C C C

In [20]:
# large_score (1 point)
tree, leaf_states = large_tree, large_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert cost == 7
print("SUCCESS: large_score test case passed!")

SUCCESS: large_score test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf9b1505f94604a23ba4dd1be5c991e5d"> D C B A H G F E 8 9 10 11 12 13 14 A T C G C C C G

In [21]:
# pair_match_states (3 points)
tree, leaf_states = pair_tree, pair_match_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'2': 'C', 'X': 'C', 'Y': 'C'}
print("SUCCESS: pair_match_states test case passed!")

SUCCESS: pair_match_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t39e6697769b84d2b878b5c09caf8a315"> Y X 2 C C

In [22]:
# pair_mismatch_states (3 points)
tree, leaf_states = pair_tree, pair_mismatch_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'2': 'A', 'X': 'A', 'Y': 'T'}
print("SUCCESS: pair_mismatch_states test case passed!")

SUCCESS: pair_mismatch_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t97e99f5eaaf644c9a5b3713451d00579"> Y X 2 T A

In [23]:
# triple_states (2 points)
tree, leaf_states = triple_tree, triple_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'4': 'C', 'Z': 'C', '3': 'C', 'X': 'A', 'Y': 'T'}
print("SUCCESS: triple_states test case passed!")

SUCCESS: triple_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb7f59ea36a9d4f1c988e317c2fc60d50"> Y X Z 3 4 T A C

In [24]:
# quartet_states (1 point)
tree, leaf_states = quartet_tree, quartet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'6': 'A', 'W': 'A', '5': 'A', 'Z': 'C', '4': 'A', 'X': 'A', 'Y': 'T'}
print("SUCCESS: quartet_states test case passed!")

SUCCESS: quartet_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfec8ff82e2024f72bfd8beba2c29cd1a"> Y X Z W 4 5 6 T A C A

In [25]:
# quartet2_states (1 point)
tree, leaf_states = quartet2_tree, quartet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'6': 'A', '5': 'A', '4': 'A', 'W': 'A', 'Z': 'C', 'X': 'A', 'Y': 'T'}
print("SUCCESS: quartet2_states test case passed!")

SUCCESS: quartet2_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6885821d950c4b56abacc83499e71e67"> Y X Z W 4 5 6 T A C A

In [26]:
# quintet_states (1 point)
tree, leaf_states = quintet_tree, quintet_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'8': 'C','7': 'C','6': 'C','V': 'C','W': 'C','Z': 'C','5': 'C','X': 'A','Y': 'T'}
print("SUCCESS: quintet_states test case passed!")

SUCCESS: quintet_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf4a36c107ca940d18ac6d5183e103f7f"> Y X Z W V 5 6 7 8 T A C C C

In [27]:
# large_states (1 point)
tree, leaf_states = large_tree, large_states
draw_tree_with_internal_labels(tree, leaf_states)
cost, states = weighted_parsimony(tree, leaf_states, purine_pyrimidine_cost_matrix)
assert states == {'14': 'C','13': 'C','12': 'C','E': 'G','11': 'C','F': 'C', '8': 'C',
                  'G': 'C','H': 'C','10': 'C','9': 'C','A': 'G','B': 'C','C': 'T','D': 'A'}
print("SUCCESS: large_states test case passed!")

SUCCESS: large_states test case passed!


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t28ddb08522264cbeb72dabb318383e58"> D C B A H G F E 8 9 10 11 12 13 14 A T C G C C C G

### Hidden tests (6 points total)

In [28]:
# p1_hidden1 (3 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [29]:
# p1_hidden2 (2 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [30]:
# p1_hidden3 (1 points)
###
### AUTOGRADER TEST - DO NOT REMOVE
###
